In [1]:
# Clipping Point Clouds with Grass Biomass Plot Shapefile
# Based on Clipping Trees function for halo paper
# PB - 02/22/2022 (lots of 2s today!)

import sys
sys.path.append('/n/home02/pbb/scripts/halo-metadata-server/')
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import laspy
import geojson
from shapely.geometry import Polygon
import time
from Classes import Collection
from Cloud_Class import Cloud, calccover
import time
import concurrent.futures
import pickle

# makes matplotlib plots big
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams.update({'font.size': 14})


In [2]:
# Function for Clipping Individual Plots/Trees
# can change featureIDcol when working with other shapefiles
# it should work with any shapefile with irregularly shaped polygon features, and some kind of id column

# Edited 11/15/21 to work in parallel
def iteratePolygons_parallel(feature=None, featureIDcol='Point_ID', epsg=32736):
    
    # Iteration time
    itertime_start = time.time()

     # subset the points to only default and ground points within the square boundary of the given shapefile feature
    idx = [(l.x <= feature.geometry.bounds[2]) &
           (l.x >= feature.geometry.bounds[0]) &
           (l.y <= feature.geometry.bounds[3]) &
           (l.y >= feature.geometry.bounds[1]) &
           (l.points.classification != 7)] 

    pointset = l.points[idx]

    # if points is not empty
    if len(pointset) > 0:

        # Make a geodataframe from the points
        points_gdf = gpd.GeoDataFrame({'time':pointset.gps_time},
                                      geometry=gpd.points_from_xy(pointset.X * l.header.x_scale + l.header.x_offset,
                                                                  pointset.Y * l.header.y_scale + l.header.y_offset),
                                      crs=f'EPSG:{epsg}')

        # get the indices of points that intersect with the polygon feature
        # use align=False to preserve the order of the index
        intersects_idx = points_gdf.intersects(feature.geometry, align=False)

        # NOW: subset your points again, this time, based on your intersection index
        points_subset = pointset[intersects_idx.values]

        # set the outf name and path
        # use the featureIDcol in the shapefile to name it 
        outf = f'{outdir}/{featureIDcol}_{feature.get(featureIDcol)}.las'
        
                    
        if len(points_subset) > 0:

            # Skipped computing metrics for this - still need to refine the buffer zone to a 0.5m plot in CC
            # COMPUTE METRICS:
            # While you have your subsetted points
            # c, p, h = calccover_parallel(points_subset)

            # Iteration time end
            itertime_end = time.time()
            totaltime = itertime_end-itertime_start

            # return points_subset, outf, totaltime, c, p, h
            return points_subset, outf, totaltime

#         # Skipped computing metrics for this - still need to refine the buffer zone to a 0.5m plot in CC
#         try:
            
#             if len(points_subset) > 0:
                
#                 # Skipped computing metrics for this - still need to refine the buffer zone to a 0.5m plot in CC
#                 # COMPUTE METRICS:
#                 # While you have your subsetted points
#                 # c, p, h = calccover_parallel(points_subset)

#                 # Iteration time end
#                 itertime_end = time.time()
#                 totaltime = itertime_end-itertime_start
                
#                 # return points_subset, outf, totaltime, c, p, h
#                 return points_subset, outf, totaltime
        
#         except:
        
#             print(f'Issue computing metrics for {featureIDcol}{feat.get(featureIDcol)}\n')


In [5]:
# # # DEFINE USER INPUTS

# # # 

# ### Before Fire
# lasdir = '/n/davies_lab/Lab/data/processed/Africa/Kruger/Kruger_June21/LowerSabie_Fires/Before_Fire/Terrasolid_Lower_Sabie_Before_fire/output_pointcloud'

# # lasinputs = [f'{lasdir}/BuffaloCamp60mCrosshatch_300mExtent_height_FOV110.las']
# lasinputs = glob.glob(f'{lasdir}/*.las')

# # # outdirectory for clipped las files
# odir = '/n/home02/pbb/scripts/halo-metadata-server/GrassBiomass/data/out/5mBuffer'
# # odir = '/n/home02/pbb/scripts/halo-metadata-server/GrassBiomass/data/out/50cmCenterPixel'

# # outdirectory for metrics - don't need it here
# # objoutdir = '/n/davies_lab/Users/pbb/collections/BuffCampAltitudeStudy/TreeMetrics_FOV110'

# # # Designate the cshapefile to use
# shpf = '/n/home02/pbb/scripts/halo-metadata-server/GrassBiomass/data/in/LowerSabie_GrassBiomassFieldPlots_5mBuffer/LowerSabie_GrassBiomassFieldPlots_5mBuffer.shp'
# # shpf = '/n/home02/pbb/scripts/halo-metadata-server/GrassBiomass/data/in/LowerSabie_GrassBiomassFieldplots_50cmCenterPixel/LowerSabie_GrassBiomassFieldPlots_50cmCenterPixel.shp'

# # # Feature ID col
# featureIDcol = 'Point_ID'

# # # MANUALLY ADD A PROJECT STRING (NOTE: HARDCODED FOR THIS PROJECT)
# projstr = 'LowerSabieBeforeFire'

# # #

# After Fire
# lasdir = '/n/davies_lab/Lab/data/processed/Africa/Kruger/Kruger_June21/LowerSabie_Fires/After_fire/Terrasolid/output_pointcloud'
lasdir = '/n/davies_lab/Users/pbb/LowerSabieGrassBiomass/data/LowerSabieAfterFire'

lasinputs = glob.glob(f'{lasdir}/*.las')

# outdirectory for clipped las files
odir = '/n/home02/pbb/scripts/halo-metadata-server/GrassBiomass/data/out/5mBuffer'
# odir = '/n/home02/pbb/scripts/halo-metadata-server/GrassBiomass/data/out/50cmCenterPixel'

# outdirectory for metrics - don't need it here
# objoutdir = '/n/davies_lab/Users/pbb/collections/BuffCampAltitudeStudy/TreeMetrics_FOV110'

# Designate the cshapefile to use
shpf = '/n/home02/pbb/scripts/halo-metadata-server/GrassBiomass/data/in/LowerSabie_GrassBiomassFieldPlots_5mBuffer/LowerSabie_GrassBiomassFieldPlots_5mBuffer.shp'
# shpf = '/n/home02/pbb/scripts/halo-metadata-server/GrassBiomass/data/in/LowerSabie_GrassBiomassFieldplots_50cmCenterPixel/LowerSabie_GrassBiomassFieldPlots_50cmCenterPixel.shp'

# Feature ID col
featureIDcol = 'Point_ID'

# MANUALLY ADD A PROJECT STRING (NOTE: HARDCODED FOR THIS PROJECT)
projstr = 'LowerSabieAfterFire'

# # # 

# # # END USER INPUTS

In [6]:
# LOOP to clip trees from each altitude flight
#  Uses concurrent futures to clip 
#  each polygon feature in the shapefile dataframe
#  in parallel

# Make a list of features from the shapefile
shpdf = gpd.read_file(shpf)
shpdf = shpdf.sort_values(featureIDcol)
features = []
for i, f in shpdf.iterrows():
    features.append(f)

# Subset features here 
# NOTE ONLY FOR TESTING PURPOSES - COMMENT OUT OTHERWISE
# features = features[0:50]
    
# For each las file inputs
for lasf in lasinputs:
    
    start_tottime = time.time()
    
    # HARDCODED ABOVE FOR GRASS BIOMASS PLOTS
    # get project string for saving below
    # projstr = lasf.split('/')[-1].split('_')[0]
    # print(f'Starting {projstr}...\n') 
    
    lasfstr = lasf.split('/')[-1]
    print(f'Starting {lasfstr}...\n') 
    
    # set the outdirectory for individual tree las files using the above projstr
    outdir = f'{odir}/{projstr}/'
    
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    
    start_load = time.time()
    
    try:
        
        # open the current las file
        l = laspy.read(lasf)

        end_load = time.time()
        print(f'Loaded {l.header.point_count/1000000} million points in {end_load-start_load} seconds.\n')

        # initiate empty list of times for each project
        times=[]

        # # Initiate dictionaries for saving treeid metrics
        # cov = {}
        # perc = {}
        # height = {}

        # set up parallel processing for each polygon feature
        with concurrent.futures.ThreadPoolExecutor(max_workers=None) as executor:
            # return the subset of points, the outlasfile name, and the processing time for each feature
            # Returned as a list in that order pnts_of_t = [pnts, of, time, coverdict, percdict, heightdict]
            for pnts_of_t_c_p_h, feat in zip(executor.map(iteratePolygons_parallel, features),
                                             features):

                # # # Write output las files
                # if there are any points to output 
                if pnts_of_t_c_p_h:

                    # Record time
                    times.append(pnts_of_t_c_p_h[2])
                    
                    # Store metrics in a nested dictionary
                    # Using featureID as the key
                    # cov[f'{featureIDcol}{feat.get(featureIDcol)}'] = pnts_of_t_c_p_h[3]
                    # perc[f'{featureIDcol}{feat.get(featureIDcol)}'] = pnts_of_t_c_p_h[4]
                    # height[f'{featureIDcol}{feat.get(featureIDcol)}'] = pnts_of_t_c_p_h[5]
                    
                    
                    try:
                        # if this file does not exist yet, make a new las file
                        if not os.path.exists(pnts_of_t_c_p_h[1]):
                            # write points to current outfile
                            with laspy.open(pnts_of_t_c_p_h[1], mode="w", header=l.header) as writer:
                                writer.write_points(pnts_of_t_c_p_h[0])
                        # else: append to the current las file
                        else:
                            # write points to current outfile
                            with laspy.open(pnts_of_t_c_p_h[1], mode="a", header=l.header) as writer:
                                writer.append_points(pnts_of_t_c_p_h[0])
                    except: 
                        
                        print(f'Unable to write lasfile: {pnts_of_t_c_p_h[1]}')
                                
        
        lasfstr = lasf.split('/')[-1]
        end_tottime = time.time()
        print(f'{lasfstr} done.\nTook {end_tottime - start_tottime} seconds to clip {len(features)} features.\n')
                              
    except ValueError as ve:
        
        lasfstr = lasf.split('/')[-1]
        print(f'Could not open {lasfstr} \n')


Starting Lower_Sabie_After_fire_PointCloud_WGS84UTM36S_000020.las...

Loaded 0.0 million points in 0.012422800064086914 seconds.

Lower_Sabie_After_fire_PointCloud_WGS84UTM36S_000020.las done.
Took 0.250974178314209 seconds to clip 98 features.

Starting Lower_Sabie_After_fire_PointCloud_WGS84UTM36S_000005.las...

Loaded 11.018666 million points in 0.666917085647583 seconds.

Lower_Sabie_After_fire_PointCloud_WGS84UTM36S_000005.las done.
Took 3.687946081161499 seconds to clip 98 features.

Starting Lower_Sabie_After_fire_PointCloud_WGS84UTM36S_000006.las...

Loaded 0.0 million points in 0.0579376220703125 seconds.

Lower_Sabie_After_fire_PointCloud_WGS84UTM36S_000006.las done.
Took 0.2851395606994629 seconds to clip 98 features.

Starting Lower_Sabie_After_fire_PointCloud_WGS84UTM36S_000004.las...

Loaded 21.457754 million points in 1.0556204319000244 seconds.

Lower_Sabie_After_fire_PointCloud_WGS84UTM36S_000004.las done.
Took 70.06173777580261 seconds to clip 98 features.

Starting L

In [39]:
# Try to automatically add plotid with laspy
# ran into issues writing the file header
# las = laspy.read(lasf)
# plotid = pnts_of_t_c_p_h[1].split('ID_')[-1].split('.')[0]
# las.plot = np.repeat(plotid, las.header.point_count).astype(np.uint16)
# las.add_extra_dim(laspy.ExtraBytesParams(name="plot", type=np.uint16))

# for i in las.point_format.dimension_names:
#     print(i)

# # write points to current outfile
# with laspy.open('test.las', mode="w", header=l.header) as writer:
#     writer.write_points(las)
    
# las.header.point_format.dimensions
# # 
